Pablo Jimeno - UPV/EHU 2016

In [ ]:
#%run 'Utilities_main.ipynb'

In [ ]:
#-------------------------------------------------------------------
def import_SZ_results(subsample_bins):
    """Import results"""    
    
    sample_metas = []
    SZ_vals = []
    SZ_err_vals = []
    
    for sub_bin in subsample_bins:
          
        rich_min, rich_max, z_min, z_max = sub_bin
        sample_tag = sub_tag(sub_bin)
        
        if (rich_min>=rich_max) or (z_min>=z_max):
            continue
    
        #-------------------------------------------------------------------
        ## Import meta & SZ data:       
        store_SZ_file = '%s/SZ_values/SZ_data_%s.pickle' % (data_dir, sample_tag)
        with open(store_SZ_file, 'rb') as handle:
            store_SZ_data = pickle.load(handle)
        
        #sub_meta : [rich_min, rich_max, z_min, z_max, N_clu, rich_mean, z_mean]
        sub_meta = store_SZ_data[0]
        N        = sub_meta[4]
        SZ       = store_SZ_data[1]/N
        SZ_err   = store_SZ_data[2]/N

        sample_metas.append(sub_meta)
        SZ_vals.append(SZ)
        SZ_err_vals.append(SZ_err)
            
    sample_metas = np.array(sample_metas)
        
    return sample_metas, SZ_vals, SZ_err_vals
    

In [ ]:
#------------------------------------------------------------------- 
def compute_SZ(SZ, SZ_err):
    """Compute SZ values."""

    def H(x): return x**4*np.exp(x)/(np.exp(x) - 1.)**2
    
    nu = np.array([100., 143., 217., 353., 545., 857.])
    X = nu/56.4
    Hx = H(X)
    Fx = SZ*Hx
    Fx_err = SZ_err*Hx
        
    return nu, Fx, Fx_err


#-------------------------------------------------------------------
def SZ_model(nu_m, Y):
   
    def H(x): return x**4*np.exp(x)/(np.exp(x) - 1.)**2
    
    def F(x): return H(x)*(x*(np.exp(x) + 1.)/(np.exp(x) - 1.) - 4.)
        
    T_CMB = 2.725
    norm_factor = 1./(1e9*planck_h/k_b/T_CMB)
    I_0 = (2.*planck_h/c**2)*(k_b*T_CMB/planck_h)**3
    
    ## SZ contribution:
    X_m = nu_m/norm_factor
    Fx_m = Y*F(X_m)
    
    return Fx_m


#-------------------------------------------------------------------
def dust_model(nu_m, A):
    
    T_dust = 20. # Dust temperature
    beta_dust = 1.6 # spectral emissivity parameter
    
    n0_dust = 217.
    
    X_m = nu_m*1e9
    X0_dust = n0_dust*1e9
    
    ## GB contribution:
    BB = (2.*planck_h*X_m**3/c**2) * 1./(np.exp(planck_h*X_m/k_b/T_dust) - 1.)*1e26 # mJy
    emissivity = (X_m/X0_dust)**beta_dust
    norm_factor = A*1e-14
    
    GB_m = norm_factor*emissivity*BB
        
    return GB_m


#-------------------------------------------------------------------
def global_model(nu_m, *params_m):
    """Compute model values using data to normalize:"""
    
    Y, A = params_m
        
    Fx_m = SZ_model(nu_m, Y)
    GB_m = dust_model(nu_m, A)
    
    SZ_m = Fx_m + GB_m
    
    return SZ_m


#-------------------------------------------------------------------
def global_model_err(nu_m, params_m, params_error):
    """Compute model values using data to normalize:"""
        
    N_rlz = 100
    realizations = []
    
    for n in range(N_rlz):
        params_rlz = [params_m[p] + np.random.normal(scale=params_error[p]) for p in range(len(params_m))]
        model_rlz = global_model(nu_m, *np.array(params_rlz))
        realizations.append(model_rlz)
    
    sigma_error = 1.
    SZ_m_err = sigma_error*np.std(realizations, axis=0)
    
    return SZ_m_err


#-------------------------------------------------------------------
def find_SZ_model(nu, Fx, Fx_err):
    """Compute model values using data to normalize:"""
        
    params_guess = np.array([1., 5.])
    
    params_fit, params_cov = optimize.curve_fit(global_model, nu, Fx, sigma=Fx_err, 
                                                p0=params_guess, absolute_sigma=True)
    
    params_error = np.sqrt(np.diagonal(params_cov))
    
    #Y_m = params_fit[0]
    #A_m = params_fit[1]
    #Y_m_err = params_error[0]
    #A_m_err = params_error[1]
    
    return params_fit, params_cov, params_error
       
    
#-------------------------------------------------------------------
def make_SZ_comparison(sample_metas, SZ_R1R2_vals, SZ_R1R2_err_vals, nr_bins=3, nz_bins=2):
    
    
    def model_SZ_signal(ax, SZ, SZ_err, meta):
        """Plot SZ signal."""
    
        nu, Fx, Fx_err = compute_SZ(SZ, SZ_err)
        params_m, params_m_cov, params_m_err = find_SZ_model(nu, Fx, Fx_err)

        Y_m = params_m[0]
        A_m = params_m[1]
        Y_m_err = params_m_err[0]
        A_m_err = params_m_err[1]

        nu_m = np.arange(1, 1000, 1.)
        Fx_m = SZ_model(nu_m, Y_m)
        GB_m = dust_model(nu_m, A_m)
        SZ_m = global_model(nu_m, *params_m)   
        SZ_m_err = global_model_err(nu_m, params_m, params_m_err)
        
        return Y_m, A_m, Y_m_err, A_m_err, nu_m, Fx_m, GB_m, SZ_m, SZ_m_err
    
    
    def export_results(Y_m, A_m, Y_m_err, A_m_err, meta):
        
        store_Y_A_data = [meta, Y_m, A_m, Y_m_err, A_m_err]
        store_Y_A_file = '%s/SZ_values/Y_A_data_%s.pickle' % (data_dir, sub_tag(meta))
        with open(store_Y_A_file, 'wb') as handle:
            pickle.dump(store_Y_A_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        
    def plot_SZ_signal(ax, SZ, SZ_err, meta):
        """Plot SZ signal."""
    
        nu, Fx, Fx_err = compute_SZ(SZ, SZ_err)
        Y_m, A_m, Y_m_err, A_m_err, nu_m, Fx_m, GB_m, SZ_m, SZ_m_err = model_SZ_signal(ax, SZ, SZ_err, meta)
        export_results(Y_m, A_m, Y_m_err, A_m_err, meta)
    
         ## Plot results:
        N = meta[4]
        label = r'$\lambda$: [%i, %i), z: [%.3f, %.3f), N: %i' % (meta[0], meta[1], meta[2], meta[3], meta[4])

        markersize = 3.

        ax.set_title(label)
        ax.set_xlim([0., 1000.])
        ax.set_xlabel(r'$\nu$ [GHz]')
        ax.set_ylim([2.*(Fx_m).min(), 15.*(Fx_m).max()])
        ax.set_ylabel(r'SZ [$\mu$K]')

        ## Data:
        ax.plot(nu_m, np.zeros(len(nu_m)), 'k-', alpha=0.3)
        ax.errorbar(nu, Fx, yerr=Fx_err, fmt='ko', marker='o', markersize=markersize, label='data')

        ## Model:
        ax.plot(nu_m, Fx_m, 'r-', alpha=0.5, label=r'SZ: Y = %.1f$\pm$%.1f' % (Y_m, Y_m_err))
        ax.plot(nu_m, GB_m, 'g-', alpha=0.5, label=r'GB: A = %.1f$\pm$%.1f' % (A_m, A_m_err))
        ax.plot(nu_m, SZ_m, 'b-', alpha=0.8)
        ax.fill_between(nu_m, (SZ_m+SZ_m_err), (SZ_m-SZ_m_err), edgecolor='b', facecolor='b', alpha=0.15,
                        label='1$\sigma$ model')

        plt.legend(loc=2, numpoints=1, frameon=False)
        
        
           
    fig = plt.figure(figsize=(nz_bins*4, nr_bins*3))
    
    for i, meta in enumerate(sample_metas):
              
        SZ = SZ_R1R2_vals[i]
        SZ_err = SZ_R1R2_err_vals[i] 
        
        ax = fig.add_subplot(nr_bins, nz_bins, nr_bins*nz_bins - (i/nz_bins+1)*nz_bins + i%nz_bins + 1)
        plot_SZ_signal(ax, SZ, SZ_err, meta)
        
    plot_file = '%s/SZ_modeling/SZ_modeling.png' % plot_dir
    plt.tight_layout()
    fig.savefig(plot_file, bbox_inches='tight')
    save_figure(fig, 'SZ_modeling')
        
    
#-------------------------------------------------------------------

#R_vals = [20, 30, 50, 300]
#z_vals = [0.100, 0.250, 0.400, 0.550]

#subsample_bins, nr_bins, nz_bins = automatic_binning(R_vals, z_vals) 

## Run code:
def run_model_code():
    sample_metas, SZ_R1R2_vals, SZ_R1R2_err_vals = import_SZ_results(subsample_bins)
    make_SZ_comparison(sample_metas, SZ_R1R2_vals, SZ_R1R2_err_vals, nr_bins, nz_bins)
   
    
try:
    run_model_code()
except:
    %run 'Analysis_SZ.ipynb'
    clear_output()
    run_model_code()

print '\nModelling completed.'